In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
import lxml

from splinter.exceptions import ElementDoesNotExist

In [ ]:
# Database Setup

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.mars_db
collection = db.articles

## Step 1 - Scraping

## NASA Mars News

In [4]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [8]:
news_title = soup.find('div', class_='content_title').a.text
news_p = soup.find('div', class_='article_teaser_body').text

print(news_title)
print(news_p)

NASA's Curiosity Rover Finds an Ancient Oasis on Mars
New evidence suggests salty, shallow ponds once dotted a Martian crater — a sign of the planet's drying climate.


## JPL Mars Space Images - Featured Image

In [28]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

image_style = soup.find('article')['style']
image_url = image_style.split("'")[1].split("'")[0]

featured_image_url = 'https://www.jpl.nasa.gov' + image_url

print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA20063-1920x1200.jpg


## Mars Weather

In [34]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

mars_weather = soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text

print(mars_weather)

InSight sol 312 (2019-10-12) low -100.4ºC (-148.7ºF) high -26.0ºC (-14.8ºF)
winds from the SSW at 4.8 m/s (10.6 mph) gusting to 19.5 m/s (43.6 mph)
pressure at 7.20 hPapic.twitter.com/t0L5YpYPah


## Mars Facts
